In [4]:
import numpy as np
import pyvista as pv
from irregular_object_packing.cat.tetrahedral_split import (
    split_2_2222,
    split_2_3331,
    split_3,
    split_4,

)

In [22]:
from itertools import combinations

from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input


tet1 = np.array([ # faces allign with axis
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
])
tet2 = np.array([ # pointy end alligns with axis
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 5],
])
tet3 = np.array([ # pointy flat equilateral
    [0, 0, 0],
    [1, 0, 0],
    [0.5, 0.866, 0],
    [0.5, 0.288, 0.1],
])
tet4 = np.array([ # 
    [0, 0, 0],
    [5, 0, 0],
    [0, 1, 0],
    [0, 0, 5],
])
def mesh_from_tet(tet:np.ndarray): 
    faces = list(combinations(tet, 3))
    return pv.PolyData(*convert_faces_to_polydata_input(faces))

    

In [32]:
from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input

colors = ["red", "green", "blue", "yellow"]
splits = [ split_2_2222, split_2_3331, split_3, split_4 ]
tets = [ tet1, tet2, ]

plotter = pv.Plotter(shape = (len(tets),len(splits)),)
for i, split in enumerate(splits):
    for j, tet in enumerate(tets):
        plotter.subplot(j,i)
        faces = split(tet)
        for k, face in enumerate(faces):
            plotter.add_mesh(pv.PolyData(*convert_faces_to_polydata_input(face)).explode(), color=colors[k])
            plotter.add_points(np.array([tet[k]]), color=colors[k])
            
        plotter.add_mesh(mesh_from_tet(tet), color="white", show_edges=True, opacity=0.8)
        
plotter.show()

/opt/homebrew/Caskroom/miniconda/base/envs/irop/lib/python3.10/site-packages/pyvista/utilities/helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


Widget(value="<iframe src='http://localhost:50395/index.html?ui=P_0x2e9e2b520_23&reconnect=auto' style='width:…

In [34]:
np.cumsum([10, 20, 30])

array([10, 30, 60])